<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 7**  
## **Генеративные модели для табличных данных и временных рядов**

---

### **1. Цель и задачи работы**

**Цель работы** — сформировать у обучающегося системное понимание современных генеративных моделей, применимых к **структурированным данным**, включая как **статические табличные датасеты**, так и **динамические временные ряды**. Работа выходит за рамки простой генерации «похожих» строк и фокусируется на **качестве синтеза, сохранении статистических свойств, временной когерентности, условной генерации и практической применимости** в задачах аугментации, балансировки, синтетического тестирования и приватности.

**Основные задачи работы:**  
1. Использовать два типа данных:  
   - **Временной ряд** из Практической работы № 1 (не менее 720 наблюдений, с инженерными признаками);  
   - **Табличный датасет без временной структуры** (например, Adult Income, Credit Card Fraud, или медицинские данные с дисбалансом классов).  
2. Реализовать и сравнить следующие категории генеративных моделей:  

   **A. Для табличных данных:**  
   - **CTGAN** (Conditional Tabular GAN) — с mode-specific normalization;  
   - **TVAE** (Tabular Variational Autoencoder) — с гауссовыми и категориальными латентами;  
   - **Gaussian Copula** — классическая параметрическая модель (базовая линия);  
   - **TabDDPM** — диффузионная модель для табличных данных;  
   - **RealTabFormer** — гибрид трансформера и дискретизации (для смешанных типов данных).  

   **B. Для временных рядов:**  
   - **TimeGAN** — генерация с сохранением временной когерентности через embedding и recovery сети;  
   - **TSGAN** — GAN с рекуррентным дискриминатором;  
   - **COT-GAN** — causal optimal transport для сохранения зависимости во времени;  
   - **TS-VAE** — вариационный автоэнкодер с рекуррентным энкодером/декодером;  
   - **DiffTime** — диффузионная модель для временных рядов.  

3. Для каждой модели:  
   - Обеспечить корректную предобработку (масштабирование, кодирование категорий, оконное нарезание для рядов);  
   - Реализовать **условную генерацию** (например, синтез только «аномальных» или «редких» наблюдений);  
   - Применить **строгое разделение**: обучение генератора — только на тренировочной части.  
4. Сгенерировать синтетические датасеты:  
   - Для табличных данных — объём, равный исходному;  
   - Для временных рядов — последовательности той же длины и количества признаков.  
5. Оценить качество синтеза по трём критериям:  
   - **Статистическая близость**: распределения признаков, корреляции, автокорреляция (для рядов);  
   - **Полезность**: обучение классификатора/регрессора на синтетических данных → оценка на реальном тесте;  
   - **Приватность**: расстояние между исходными и синтетическими точками (k-anonymity, distance to nearest neighbour).  
6. Проанализировать:  
   - **Время генерации и обучения**;  
   - **Стабильность обучения** (коллапс режимов, осцилляции loss);  
   - **Интерпретируемость латентного пространства** (для VAE и Diffusion).  
7. Исследовать практические применения:  
   - Аугментация несбалансированного класса;  
   - Имитация «что если»-сценариев (например, изменение температуры → изменение спроса);  
   - Создание приватного синтетического датасета для публикации.  
8. Разработать модульный фреймворк на PyTorch с единым интерфейсом: `fit()`, `sample(n)`, `conditional_sample(condition)`.  
9. Опубликовать воспроизводимый пайплайн: код, синтетические датасеты, метрики качества, визуализации.

---

### **2. Теоретические предпосылки**

Генеративные модели для табличных данных и временных рядов решают фундаментальную проблему: **как смоделировать сложное совместное распределение \( p(x_1, x_2, ..., x_d) \)** в условиях смешанных типов данных, дисбаланса, разреженности и, в случае рядов, **временной зависимости**.

Классические подходы (например, **Gaussian Copula**) предполагают параметрические распределения и не справляются с мультимодальностью. Современные модели отказываются от этих допущений:

- **GAN-подходы** (CTGAN, TimeGAN) обучаются через минимакс-игру между генератором и дискриминатором. **TimeGAN** дополнительно вводит **embedding** и **recovery** сети, чтобы сохранить как статистические, так и временные свойства.  
- **VAE** (TVAE, TS-VAE) максимизируют нижнюю границу правдоподобия, обеспечивая **гладкое латентное пространство**, пригодное для интерполяции и условной генерации.  
- **Диффузионные модели** (TabDDPM, DiffTime) последовательно добавляют и удаляют шум, демонстрируя **высокое качество синтеза**, но за счёт вычислительной стоимости.  
- **Flow-based модели** (не включены в основной список из-за сложности для дискретных данных) используют обратимые преобразования для точного моделирования плотности.

Ключевое различие между **табличными** и **временными** генеративными моделями — в обработке **последовательной структуры**. В то время как CTGAN генерирует строки независимо, **TimeGAN** и **COT-GAN** явно моделируют переходы \( x_t \rightarrow x_{t+1} \), что критично для сохранения динамики.

Важно: **цель генерации — не точное копирование, а сохранение полезных свойств данных**. Хороший синтетический датасет должен позволять обучать модели, которые работают на реальных данных.

---

### **3. Порядок выполнения работы**

#### **3.1. Подготовка данных**

- **Табличные данные**: обработка категорий (label encoding или embedding), масштабирование числовых признаков;  
- **Временные ряды**: преобразование в последовательности фиксированной длины; сохранение временной метки не требуется;  
- Для условной генерации — выделение целевой переменной или редкого класса.

#### **3.2. Реализация моделей**

- Использование библиотек: **SDV (Synthetic Data Vault)**, **ydata-synthetic**, или воспроизведение по статьям;  
- Для кастомных моделей (COT-GAN, DiffTime) — реализация на PyTorch;  
- Единый интерфейс: `model.fit(X_train)`, `model.sample(n)`, `model.conditional_sample(condition, n)`.

#### **3.3. Генерация и оценка**

- Для каждой модели — генерация датасета того же объёма, что и исходный;  
- **Метрики качества**:  
  - *Статистика*: Kolmogorov–Smirnov для числовых, точность совпадений для категориальных;  
  - *Корреляции*: матрица корреляций (Pearson/Spearman) — сравнение с оригиналом;  
  - *Для рядов*: ACF/PACF синтетического ряда vs. реального;  
  - *Полезность*: обучение LightGBM на синтетике → MAE/RMSE на реальном тесте;  
  - *Приватность*: среднее расстояние от синтетической точки до ближайшей реальной.

#### **3.4. Практические сценарии**

- **Аугментация**: синтез 1000 дополнительных «мошеннических» транзакций → обучение → оценка F1 на тесте;  
- **Имитация**: в TimeGAN — генерация сценария «повышенной температуры» → анализ реакции целевой переменной;  
- **Публикация**: подготовка анонимизированного синтетического датасета для открытого репозитория.

#### **3.5. Сравнение и выводы**

- Сравнительная таблица: «Модель × Тип данных × Статистика × Полезность × Время генерации × Устойчивость»;  
- Рекомендации:  
  — **CTGAN/TVAE** — для быстрой аугментации табличных данных;  
  — **TimeGAN** — для синтеза реалистичных временных рядов;  
  — **Diffusion** — если качество критично, а время — нет.

#### **3.6. Публикация**

- GitHub-репозиторий с кодом и сгенерированными данными;  
- Синтетический датасет — в формате CSV/Parquet с описанием;  
- Визуализации: распределения, ACF, scatter plots «реальные vs. синтетические».

---

### **4. Дополнительные исследовательские задания**

1. **Оценка утечки данных**: можно ли восстановить исходные наблюдения из синтетических?  
2. **Генерация с ограничениями**: например, «температура > 30°C, но спрос < 100»;  
3. **Перенос обучения**: обучить TimeGAN на одном ряду, сгенерировать другой — насколько это возможно?  
4. **Сравнение с простой аугментацией**: SMOTE vs. CTGAN для несбалансированных данных;  
5. **Влияние размера обучающей выборки**: как качество синтеза зависит от N?

---

### **5. Требования к отчёту**

Отчёт оформляется по ГОСТ 7.32–2017 и включает:  
1. **Введение** — мотивация генеративного моделирования для структурированных данных;  
2. **Методология** — описание моделей, данных, метрик качества;  
3. **Результаты** — таблицы, графики распределений, ACF, метрики полезности;  
4. **Обсуждение** — когда генерация оправдана, а когда — избыточна; сравнение подходов;  
5. **Заключение** — рекомендации по выбору модели в зависимости от задачи (аугментация, приватность, имитация);  
6. **Список литературы** — оригинальные статьи по TimeGAN, CTGAN, TabDDPM и др.;  
7. **Приложения** — фрагменты кода, примеры синтетических данных, скриншоты визуализаций.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Реализованы ≥6 моделей; оценка по всем трём критериям (статистика, полезность, приватность); условная генерация; практические сценарии; модульный код; публикация; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 4–5 моделей; есть базовая оценка качества; отсутствуют лишь отдельные элементы (например, условная генерация). |
| **Удовлетворительно** | Реализованы 2–3 модели (например, CTGAN и TimeGAN); есть синтетические данные и простые метрики. |
| **Неудовлетворительно** | Генерация без оценки качества; нет разделения данных; отчёт не предоставлен. |
